<a href="https://colab.research.google.com/github/paruliansaragi/DL-Notebooks/blob/master/RFG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install fastai==0.7.0

In [0]:
import pandas as pd

In [0]:
test = pd.read_csv('sample_data/california_housing_test.csv')
train = pd.read_csv('sample_data/california_housing_train.csv')

In [0]:
train.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0


In [0]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from fastai.imports import *
from fastai.structured import *
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display
from sklearn import metrics

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [0]:
def display_all(df):
  with pd.option_context("display.max_rows", 1000):
    with pd.option_context("display.max_columns", 1000):
      display(df)#stops pandas from truncating with elipses
      
display_all(train.tail().transpose())

,2995,2996,2997,2998,2999
longitude,-119.860,-118.1400,-119.7000,-117.1200,-119.6300
latitude,34.420,34.0600,36.3000,34.1000,34.4200
housing_median_age,23.000,27.0000,10.0000,40.0000,42.0000
total_rooms,1450.000,5257.0000,956.0000,96.0000,1765.0000
total_bedrooms,642.000,1082.0000,201.0000,14.0000,263.0000
population,1258.000,3496.0000,693.0000,46.0000,753.0000
households,607.000,1036.0000,220.0000,14.0000,260.0000
median_income,1.179,3.3906,2.2895,3.2708,8.5608
median_house_value,225000.000,237200.0000,62000.0000,162500.0000,500001.0000


In [0]:
train.median_house_value = np.log(train.median_house_value)

In [0]:
df, y, nas = proc_df(train, 'median_house_value')

In [0]:
m = RandomForestRegressor(n_jobs=-1)
m.fit(df, y)
m.score(df, y)

0.9690458887473821

In [0]:
train.shape, test.shape, y.shape

((17000, 9), (3000, 9), (17000,))

In [0]:
def split_vals(a, n): return a[:n].copy(), a[n:].copy()

number_valid = 3000
number_train = len(df)-number_valid
raw_train, raw_valid = split_vals(train, number_train)
X_train, X_valid = split_vals(df, number_train)
y_train, y_valid = split_vals(y, number_train)

X_train.shape, X_valid.shape, y_train.shape, y_valid.shape


((14000, 8), (3000, 8), (14000,), (3000,))

In [0]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())
def print_score(m):
    res = [rmse(m.predict(X_train), y_train),
           rmse(m.predict(X_valid), y_valid),
           m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)
m = RandomForestRegressor(n_jobs=-1)
%time m.fit(X_train, y_train)
print_score(m)

CPU times: user 1.03 s, sys: 4.06 ms, total: 1.04 s
Wall time: 612 ms
[0.10021382311521283, 0.42830888254920985, 0.967985480877531, 0.48521255764616716]


In [0]:
set_rf_samples(2000)

In [0]:
m = RandomForestRegressor(n_estimators=40, min_samples_leaf=3, max_features=0.5, n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
print_score(m)

[0.229273305637899, 0.3953950174371333, 0.8324290595994469, 0.5612913813505516, 0.7981072203650039]


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)
print_score(regr)

[0.33892059718436224, 0.3425390724208223, 0.6338258791055646, 0.6707437583210949]
